# **Lab 04** - ISA Assembler Design (Part 2)
In this lab, your task is to utilize the **pre-processed assembly code** obtained in part 1 and convert it to **machine code**.

**TODO**
* Complete tasks
* Save code to a `.bin` file

_Note:_ Some portions of the code have already been implemented for you, such as reading the input file (`_out1.txt`), adjusting the `.txt` file format into a list of instructions, printing the processed instructions, and getting the instruction's `format`, `opcode`, `funct3`, and `funct7` based on its name. Also, assume that inputs for a task are outputs from the previous task.

In [7]:
import re
import csv

## Function to read .txt file with pre-processed assembly code
def read_processed(filename):
    '''read each line from a file'''
    asm_inst = list()
    with open(filename, 'r') as f:
        for line in f:
            asm_inst.append([(int(arg) if re.fullmatch("[+-]?[0-9]+",arg) else arg) for arg in line.split()])
    return asm_inst

## Function to print the instructions
def print_asm_inst(inst_asm):
    '''prints list of instructions'''
    print("Assembly Instructions:")
    if len(inst_asm) == 0:
        print(None)
    else:
        for line in inst_asm:
            print(line)

## Read csv file containing the format for each type of instruction
def get_isa(filename):
        isa = dict()
        with open(filename, newline='') as f:
            data = csv.reader(f)
            header = next(data)
            for row in data:
                isa[row[0]] = {header[1]: row[1], header[2]: row[2], header[3]: row[3], header[4]: row[4]}
        return isa

isa = get_isa('rv32im_isa.csv')

## Gets the encoding for the respective instruction
def get_inst_format(inst_name, isa=isa):
    '''gets the instruction's format based on its name'''
    try:
        val = isa[inst_name]['format']
    except:
        raise KeyError(f"Invalid instruction: {inst_name}")
    if val == 'None':
        raise ValueError(f"Instruction '{inst_name}' does not have a format")
    return val

def get_inst_opcode(inst_name, isa=isa):
    '''gets the instruction's opcode based on its name'''
    try:
        val = isa[inst_name]['opcode']
    except:
        raise KeyError(f"Invalid instruction: {inst_name}")
    if val == 'None':
        raise ValueError(f"Instruction '{inst_name}' does not have an opcode")
    return val

def get_inst_funct3(inst_name, isa=isa):
    '''gets the instruction's funct3 based on its name'''
    try:
        val = isa[inst_name]['funct3']
    except:
        raise KeyError(f"Invalid instruction: {inst_name}")
    if val == 'None':
        raise ValueError(f"Instruction '{inst_name}' does not have 'funct3'")
    return val

def get_inst_funct7(inst_name, isa=isa):
    '''gets the instruction's funct7 based on its name'''
    try:
        val = isa[inst_name]['funct7']
    except:
        raise KeyError(f"Invalid instruction: {inst_name}")
    if val == 'None':
        raise ValueError(f"Instruction '{inst_name}' does not have 'funct7'")
    return val



## Convert int to bin (signed 2's C or unsigned)
def get_2c_binary(integer:int, bits=32, is_signed=True):
        '''converts integert to binary size bits.
        If is_signed=True, then converts to 2's Complement binary;
        Otherwise, converts to unsigned binary'''
        limit = 2**(bits)
        if is_signed:
            if (int(integer) < -limit/2) or (int(integer) >= limit/2):
                raise ValueError(f"Value outside of range: {integer}.\nMust be between [{-limit/2}, {limit/2}).")
        else:
            if (int(integer) < 0) or (int(integer) >= limit):
                raise ValueError(f"Value outside of range: {integer}.\nMust be between [0, {limit}).")
        # if no issues:
        return format(int(integer) & (limit-1), f"0{bits}b")

In [8]:
 # list to store instructions
inst_asm = []

## reads assembly code and stores it in list of lists 'inst_asm' where axis 0 (rows) corresponds 
## to each line in the file and axis 1 (columns) corresponds to each argument in that instruction
filename = "example1_out1.txt"
inst_asm = read_processed(filename)
# inst_asm = [[arg for arg in line.split()] for line in inst_asm]
print_asm_inst(inst_asm)

Assembly Instructions:
['lw', 7, 9, 0]
['addi', 10, 0, 10]
['addi', 11, 0, 20]
['bge', 10, 11, 12]
['add', 12, 11, 0]
['jal', 1, 12]
['add', 12, 10, 0]
['jal', 1, 4]
['sw', 9, 10, 0]


## **Task 1**
<span style="color:black; background-color:#C5E0B4; border: 1px solid; padding: 5px;">Implement a function that converts the above instructions from `inst_asm` into machine code.</span>

You will need the following functions: 
- `get_inst_format()` ✓
- `get_inst_opcode()` 
- `get_inst_funct3()`
- `get_inst_funct7()`
- `get_2c_binary()`

_**Note:** python indexing is <u>backwards</u> compared to how we conventionally index in hardware design. This is important to know when implementing the `imm` (immediate) field._

_**Note:** since immediate values can be <u>negative</u>, we must account for this when converting integers._

In [9]:
def get_machine_code(inst_asm):
    '''converts the assembly code to machine code'''
    inst_bin = [] # holds the final result after calling the appropriate functions

    for line in inst_asm:
        inst_name = line[0]
        match (get_inst_format(inst_name)):

            # -- enter your code here

            # R-type
            case 'R':
                # get fields
                opcode = get_inst_opcode(inst_name)
                ## ... add the other fields here ... ##
                rs1 = get_2c_binary(int(line[2]), 5)
                funct3 = get_inst_funct3(inst_name)
                funct7 = get_inst_funct7(inst_name)
                rd = get_2c_binary(int(line[1]), 5)
                rs2 = get_2c_binary(int(line[3]), 5)
                
                # assemble instruction
                code = funct7 + rs2 + rs1 + funct3 + rd + opcode
                inst_bin.append(code) # append machine code to result


            # I-type
            case 'I':
                # get fields
                opcode = get_inst_opcode(inst_name)
                ## ... add the other fields here ... ##
                imm = get_2c_binary(int(line[3]), 12)
                rs1 = get_2c_binary(int(line[2]), 5)
                funct3 = get_inst_funct3(inst_name)
                rd = get_2c_binary(int(line[1]), 5)
                
                # assemble instruction
                code = imm + rs1 + funct3 + rd + opcode
                inst_bin.append(code) # append machine code to result
                
            ## ... ##
            
            # S-type
            case 'S':
                # get fields
                opcode = get_inst_opcode(inst_name)
                ## ... add the other fields here ... ##
                rs1 = get_2c_binary(int(line[1]), 5)
                funct3 = get_inst_funct3(inst_name)
                rs2 = get_2c_binary(int(line[2]), 5)
                imm = get_2c_binary(int(line[3]), 12)
                
                # assemble instruction
                code = imm[:7] + rs2 + rs1 + funct3 + imm[7:] + opcode
                inst_bin.append(code) # append machine code to result              
            ## ... ##
            
            # B-type
            case 'B':
                # get fields
                opcode = get_inst_opcode(inst_name)
                ## ... add the other fields here ... ##
                rs1 = get_2c_binary(int(line[1]), 5)
                funct3 = get_inst_funct3(inst_name)
                rs2 = get_2c_binary(int(line[2]), 5)
                imm = get_2c_binary(int(line[3]), 12)
    
                # assemble instruction
                code = imm[:7] + rs2 + rs1 + funct3 + imm[7:] + opcode
                inst_bin.append(code) # append machine code to result
            ## ... ##
            
            # J-type
            case 'J':
                # get fields
                opcode = get_inst_opcode(inst_name)
                ## ... add the other fields here ... ##
                rd = get_2c_binary(int(line[1]), 5)
                imm = get_2c_binary(int(line[2]), 20)
                
                # assemble instruction
                code = imm + rd + opcode
                inst_bin.append(code) # append machine code to result
            ## ... ##
            
            
            # -- end your code here 

            
            case _: # Other (default)
                code = "0"*32 # assemble instruction: NOP
                inst_bin.append(code) # append machine code to result

    return inst_bin

## -- check your output by uncommenting the lines below -- ##
inst_bin = get_machine_code(inst_asm)
print_asm_inst(inst_bin)

Assembly Instructions:
00000000000001001000001110000011
00000000101000000000010100010011
00000001010000000000010110010011
00000000101101010101011001100011
00000000000001011000011000110011
00000000000000001100000011101111
00000000000001010000011000110011
00000000000000000100000011101111
00000000101001001010000000100011


## **Task 2**
<span style="color:black; background-color:#C5E0B4; border: 1px solid; padding: 5px;">Implement a function to save the processed assembly code to a `.bin` file.</span>

In [10]:
def save_bin(inst_bin, filename):
    # -- enter your code here
    '''save each machine code to a file'''
    with open(filename, 'w') as f:
        for code in inst_bin:
            f.write(code + '\n')
    print(f"Saved machine code to: {filename}")

    # -- end your code here 

## -- save your final output to a .bin file by uncommenting the lines below -- ##
save_bin(inst_bin, filename[:-5]+"2.bin")

Saved machine code to: example1_out2.bin
